In [1]:
#!pip3 install -q pyspark

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
#from langdetect import detect
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
os.listdir('.')

['name.basics.tsv',
 'title.basics.tsv',
 'title.crew.tsv',
 'title.ratings.tsv',
 '.ipynb_checkpoints',
 'title.akas.tsv',
 'title.episode.tsv',
 'Untitled.ipynb',
 'title.principals.tsv']

In [4]:
import pyspark as spark
import pyspark.sql.functions as F
import pyspark.ml as ml
import pyspark.mllib as mllib
from pyspark.sql.types import *

In [5]:
sc = spark.SparkContext()
sql = spark.sql.SQLContext(sc)

In [6]:
df = sql.read.csv('title.basics.tsv', sep=r'\t', header=True)
# df.show()

In [7]:
#df.select(df['titleType']=='movie').show(2)
df = df.drop('endYear')
df = df.filter((df.titleType=='movie')|(df.titleType=='tvMovie')) #.show(3)
df = df.filter(df.startYear>1950)
df.show(3)

+---------+---------+--------------------+--------------------+-------+---------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+--------------+--------------------+
|tt0015414|    movie|La tierra de los ...|La tierra de los ...|      0|     2000|            60|                  \N|
|tt0015724|    movie|       Dama de noche|       Dama de noche|      0|     1993|           102|Drama,Mystery,Rom...|
|tt0016906|    movie|          Frivolinas|          Frivolinas|      0|     2014|            80|      Comedy,Musical|
+---------+---------+--------------------+--------------------+-------+---------+--------------+--------------------+
only showing top 3 rows



In [8]:
origins = sql.read.csv('title.akas.tsv', sep=r'\t', header=True)
origins.show(2)

+---------+--------+----------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|     title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+----------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|Карменсіта|    UA|      \N|imdbDisplay|           \N|              0|
|tt0000001|       2|Carmencita|    DE|      \N|         \N|literal title|              0|
+---------+--------+----------+------+--------+-----------+-------------+---------------+
only showing top 2 rows



In [9]:
origins.select('language').distinct().collect()[:10]

[Row(language='en'),
 Row(language='cr'),
 Row(language='vi'),
 Row(language='ne'),
 Row(language='ps'),
 Row(language='qbo'),
 Row(language='sl'),
 Row(language='ro'),
 Row(language='uz'),
 Row(language='haw')]

In [10]:
origins = origins.filter(origins.language=='en')
origins.count()

244250

In [20]:
origins = origins.withColumnRenamed("titleId","tconst")

In [21]:
raitings = sql.read.csv('title.ratings.tsv', sep=r'\t', header=True)

In [22]:
raitings = raitings.filter(raitings['numVotes']>2500)

In [23]:
full_df=origins.join(raitings, on=['tconst'], how='right')

In [26]:
full_df = full_df.dropna()

In [27]:
full_df=full_df.join(df, on=['tconst'], how='right')

In [31]:
full_df = full_df.dropna()

In [32]:
full_df.write.csv('filtered_imdb.csv',header=True)